In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.
print ('Reading train!')
train = pd.read_csv('/data/arpit.goel/18_InventoryPrediction/02.ExtractedData/train.csv',
                    usecols=['Agencia_ID',
                                  'Ruta_SAK',
                                  'Cliente_ID',
                                  'Producto_ID',
                                  'Demanda_uni_equil'],
                    dtype={'Agencia_ID': 'uint16',
                                      'Ruta_SAK': 'uint16',
                                      'Cliente_ID': 'int32',
                                      'Producto_ID': 'uint16',
                                      'Demanda_uni_equil': 'float32'})
print ('Train read!')

Reading train!
Train read!


In [4]:
print ('Estimating means features')
train['Demanda_uni_equil'] = train['Demanda_uni_equil'].apply(lambda x: 1.005*np.log1p(x + 0.01) - 0.005)
mean = train['Demanda_uni_equil'].mean()
print ('Transformed DuE')
train['MeanP'] = train.groupby('Producto_ID')['Demanda_uni_equil'].transform(np.mean).astype('float32')
print ('Got MeanP')
train['MeanC'] = train.groupby('Cliente_ID')['Demanda_uni_equil'].transform(np.mean).astype('float32')
print ('Got MeanC')
train['MeanPA'] = train.groupby(['Producto_ID',
                                 'Agencia_ID'])['Demanda_uni_equil'].transform(np.mean).astype('float32')
print ('Got MeanPA')
train['MeanPR'] = train.groupby(['Producto_ID',
                                 'Ruta_SAK'])['Demanda_uni_equil'].transform(np.mean).astype('float32')
print ('Got MeanPR')
train['MeanPCA'] = train.groupby(['Producto_ID',
                                  'Cliente_ID',
                                  'Agencia_ID'])['Demanda_uni_equil'].transform(np.mean).astype('float32')
print ('Got MeanPCA')
train.drop_duplicates(subset=['Agencia_ID', 'Cliente_ID', 'Ruta_SAK', 'Producto_ID'], inplace=True)

print ('Dropping duplicates!')


Estimating means features
Transformed DuE
Got MeanP
Got MeanC
Got MeanPA
Got MeanPR
Got MeanPCA
Dropping duplicates!


In [5]:
MPCA = train.loc[:, ['Producto_ID', 'Agencia_ID', 'Cliente_ID', 'MeanPCA']].drop_duplicates(subset=['Agencia_ID', 'Cliente_ID', 'Producto_ID'])
print ('1')
MPA = train.loc[:, ['Producto_ID', 'Agencia_ID', 'MeanPA']].drop_duplicates(subset=['Agencia_ID', 'Producto_ID'])
print ('2')
MC = train.loc[:, ['Cliente_ID','MeanC']].drop_duplicates(subset=['Cliente_ID'])
print ('3')
MP = train.loc[:, ['Producto_ID', 'MeanP']].drop_duplicates(subset=['Producto_ID'])
print ('4')
MPR = train.loc[:, ['Producto_ID', 'Ruta_SAK', 'MeanPR']].drop_duplicates(subset=['Ruta_SAK', 'Producto_ID'])
print ('5')


1
2
3
4
5


In [6]:
print ('Reading Test')
test = pd.read_csv('/data/arpit.goel/18_InventoryPrediction/02.ExtractedData/test.csv',
                   usecols=['Agencia_ID',
                              'Ruta_SAK',
                              'Cliente_ID',
                              'Producto_ID',
                            'id'],
                   dtype={'Agencia_ID': 'uint16',
                                      'Ruta_SAK': 'uint16',
                                      'Cliente_ID': 'int32',
                                      'Producto_ID': 'uint16',
                                      'id': 'int32'})
print ('Test read!')
print ('Merging!')
test = test.merge(MPCA,
                  how='left',
                  on=['Producto_ID', 'Agencia_ID', 'Cliente_ID'],
                  copy=False)
print ('PCA merged!')
test = test.merge(MPA,
                  how='left',
                  on=['Producto_ID', 'Agencia_ID'],
                  copy=False)
print ('PA merged!')
test = test.merge(MC,
                  how='left',
                  on=['Cliente_ID'],
                  copy=False)
print ('C merged!')
test = test.merge(MP,
                  how='left',
                  on=['Producto_ID'],
                  copy=False)
print ('P merged')
test = test.merge(MPR,
                  how='left',
                  on=['Producto_ID', 'Ruta_SAK'],
                  copy=False)
print ('PR merged')

print ('Merging done!')
print (test.shape)


Reading Test
Test read!
Merging!
PCA merged!
PA merged!
C merged!
P merged
PR merged
Merging done!
(6999251, 10)


In [7]:
test.loc[:, 'Demanda_uni_equil'] = test.loc[:, 'MeanPCA'].apply(np.expm1)*0.723 +\
                            test.loc[:, 'MeanPR'].apply(np.expm1)*0.194 + 0.091
indeks = test['Demanda_uni_equil'].isnull()
test.loc[indeks, 'Demanda_uni_equil'] = test.loc[indeks, 'MeanPR'].apply(np.expm1)*0.746 + 0.17
indeks = test['Demanda_uni_equil'].isnull()
test.loc[indeks, 'Demanda_uni_equil'] = test.loc[indeks, 'MeanC'].apply(np.expm1)*0.82 + 0.86
indeks = test['Demanda_uni_equil'].isnull()
test.loc[indeks, 'Demanda_uni_equil'] = test.loc[indeks, 'MeanPA'].apply(np.expm1)*0.54 + 0.8
indeks = test['Demanda_uni_equil'].isnull()
test.loc[indeks, 'Demanda_uni_equil'] = test.loc[indeks, 'MeanP'].apply(np.expm1)*0.44 + 1
indeks = test['Demanda_uni_equil'].isnull()
test.loc[indeks, 'Demanda_uni_equil'] = np.expm1(mean)
test.loc[:, ['id', 'Demanda_uni_equil']].to_csv('/data/arpit.goel/18_InventoryPrediction/05.Submissions/submission_public_script1.csv', index=False)

In [8]:
#train['MeanP'] = train.groupby('Producto_ID')['Demanda_uni_equil'].transform(np.mean).astype('float32')
train.groupby('Producto_ID')['Demanda_uni_equil'].transform(np.mean).astype('float32')

0           1.087151
1           1.102295
2           1.154137
3           1.521830
4           1.476789
5           1.502108
6           1.279126
7           1.743002
8           1.732456
9           2.520366
10          1.801308
11          1.423805
12          2.140671
13          1.852354
14          1.747349
15          2.176181
16          1.484711
17          1.469339
18          2.258018
19          1.605822
20          1.756621
21          1.213807
22          2.014619
23          1.716188
24          1.557080
25          1.332082
26          1.557080
27          0.959698
28          1.332082
29          1.157976
              ...   
74180427    1.203060
74180430    1.143278
74180431    1.711345
74180432    1.329460
74180433    1.297095
74180434    1.154474
74180435    1.667817
74180436    1.035590
74180437    1.300450
74180439    1.667817
74180440    1.439111
74180441    1.143278
74180442    1.711345
74180443    1.154474
74180444    1.453260
74180445    1.667817
74180446    1